In [1]:
import pandas as pd
import re
import time
import numpy as np
import math
import gc
import pickle
import os
from sklearn.metrics import roc_auc_score,log_loss,f1_score
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import time
import lightgbm as lgb
from scipy.stats import mode
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA,NMF
import networkx as nx


df_train_operation = pd.read_csv("../data/operation_TRAIN_new.csv")
df_train_transaction = pd.read_csv("../data/transaction_TRAIN_new.csv")
df_label = pd.read_csv("../data/new_label.csv")#trick修改后的label
df_test_operation = pd.read_csv("../data/test_operation_round2.csv")
df_test_transaction = pd.read_csv("../data/test_transaction_round2.csv")

# 提交样例.csv
sub = pd.read_csv("../data/new/submit_example.csv")[["UID"]]

E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# utils

def analyse(data,name,label='label'):
    xx = data[["UID",label]+name].drop_duplicates()
    result = xx.groupby(name)[label].agg({'count':'count',
                                              'sum':'sum'}).reset_index()
    result['rate'] = result['sum']/(result['count']+20)
    return result

def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 'TC_AUC',0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3,True




def get_feature(op,trans,label):
    
    op["ip1_sup_add_ip2_sub"] = (op["ip1_sub"] + op["ip2_sub"]).astype(str)
    op["os_version"] = op["os"].astype(str) + op["version"].astype(str)

    col = [i for i in op.columns[2:]if i not in ["time"] ]
    for feature in col:
        label = label.merge(op.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
        label =label.merge(op.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
        
    label =label.merge(op.groupby(['UID'])["success"].mean().reset_index(),on='UID',how='left')
    
    
    col = [i for i in trans.columns[2:] if i not in ["time"] ]
    for feature in col:
        if trans_train[feature].dtype == 'object':
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
        else:
            print(feature)
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].max().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].min().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].sum().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].mean().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].std().reset_index(),on='UID',how='left')
            
    op["null_sum"] = op.isnull().sum(axis=1)
    trans["null_sum"] = trans.isnull().sum(axis=1)
    
    tmp = op.groupby(['UID'])['null_sum'].agg({'min_null_num': 'min',
                                                                 'mean_null_num': 'mean',
                                                                 'median_null_num': 'median',
                                                                 'max_null_num': 'max',
                                                                }).reset_index().fillna(0)
    label = pd.merge(label,tmp,on="UID",how="left")
    
    tmp = trans.groupby(['UID'])['null_sum'].agg({'min_null_num': 'min',
                                                                 'mean_null_num': 'mean',
                                                                 'median_null_num': 'median',
                                                                 'max_null_num': 'max',
                                                                }).reset_index().fillna(0)
    label = pd.merge(label,tmp,on="UID",how="left")
    
    t1 = trans[["UID","trans_type2"]]
    t1["trans_type2_cnt"] = 1
    tmp = t1.groupby(["UID","trans_type2"])["trans_type2_cnt"].count().reset_index()
    tmp = pd.pivot_table(tmp,index="UID",columns="trans_type2",values="trans_type2_cnt").reset_index().fillna(0)
    label = pd.merge(label,tmp,on="UID",how="left")
    
    t1 = trans[["UID", "trans_type1"]]
    t1["trans_type1_cnt"] = 1
    tmp = t1.groupby(["UID", "trans_type1"])["trans_type1_cnt"].count().reset_index()
    tmp = pd.pivot_table(tmp, index="UID", columns="trans_type1", values="trans_type1_cnt").reset_index().fillna(0)
    label = pd.merge(label, tmp, on="UID", how="left")
    
     
    print("df shape",label.shape)
    
    return label

def get_magic_fea(df_train,df_test,label,sub,col):
    for i in col:
        print(i)
        df_train[i] = df_train[i].fillna("missing").astype(str)
        df_test[i] = df_test[i].fillna("missing").astype(str)
        common = [x for x in list(set(df_train[i].tolist()).intersection(set(df_test[i].tolist()))) if x not in ["missing"]]
        df_train["magic_"+i] = df_train[i].isin(common).astype(int)
        df_test["magic_"+i] = df_test[i].isin(common).astype(int)
        label =label.merge(df_train.groupby(['UID'])["magic_"+i].mean().reset_index(),on='UID',how='left')
        sub =sub.merge(df_test.groupby(['UID'])["magic_"+i].mean().reset_index(),on='UID',how='left')
    return label,sub
    
def gen_leak_2(df):
    res = df[["UID"]].drop_duplicates()
    
    tmp = df.groupby(["ip1"])["UID"].nunique().reset_index()
    tmp.columns =["ip1","ip1_nunique"]
    tmp.sort_values(by="ip1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","ip1"]].drop_duplicates(),tmp,on="ip1",how="left")[["UID","ip1_nunique"]]
    tmp = tmp.groupby(['UID'])['ip1_nunique'].agg({'max_ip1_nunique_num_2': 'max','std_ip1_nunique_num_2': 'std'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left") 
     
    tmp = df.groupby(["ip1_sub"])["UID"].nunique().reset_index()
    tmp.columns =["ip1_sub","ip1_sub_nunique"]
    tmp.sort_values(by="ip1_sub_nunique",ascending=False)
    tmp = pd.merge(df[["UID","ip1_sub"]].drop_duplicates(),tmp,on="ip1_sub",how="left")[["UID","ip1_sub_nunique"]]
    tmp = tmp.groupby(['UID'])['ip1_sub_nunique'].agg({'max_ip1_sub_nunique_num_2': 'max','std_ip2_nunique_num_2': 'std'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device_code1"])["merchant"].nunique().reset_index()
    tmp.columns =["device_code1","device_code1_nunique"]
    tmp.sort_values(by="device_code1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code1"]].drop_duplicates(),tmp,on="device_code1",how="left")[["UID","device_code1_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code1_nunique'].agg({'max_device_code1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device_code1"])["ip1"].nunique().reset_index()
    tmp.columns =["device_code1","device_code1_nunique"]
    tmp.sort_values(by="device_code1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code1"]].drop_duplicates(),tmp,on="device_code1",how="left")[["UID","device_code1_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code1_nunique'].agg({'max_device_code1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device_code1"])["geo_code"].nunique().reset_index()
    tmp.columns =["device_code1","device_code1_nunique"]
    tmp.sort_values(by="device_code1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code1"]].drop_duplicates(),tmp,on="device_code1",how="left")[["UID","device_code1_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code1_nunique'].agg({'max_device_code1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
      
    tmp = df.groupby(["device_code1"])["UID"].nunique().reset_index()
    tmp.columns =["device_code1","device_code1_nunique"]
    tmp.sort_values(by="device_code1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code1"]].drop_duplicates(),tmp,on="device_code1",how="left")[["UID","device_code1_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code1_nunique'].agg({'max_device_code1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device_code2"])["UID"].nunique().reset_index()
    tmp.columns =["device_code2","device_code2_nunique"]
    tmp.sort_values(by="device_code2_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code2"]].drop_duplicates(),tmp,on="device_code2",how="left")[["UID","device_code2_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code2_nunique'].agg({'max_device_code2_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device_code3"])["UID"].nunique().reset_index()
    tmp.columns =["device_code3","device_code3_nunique"]
    tmp.sort_values(by="device_code3_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code3"]].drop_duplicates(),tmp,on="device_code3",how="left")[["UID","device_code3_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code3_nunique'].agg({'max_device_code3_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["acc_id1"])["UID"].nunique().reset_index()
    tmp.columns =["acc_id1","acc_id1_nunique"]
    tmp.sort_values(by="acc_id1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","acc_id1"]].drop_duplicates(),tmp,on="acc_id1",how="left")[["UID","acc_id1_nunique"]]
    tmp = tmp.groupby(['UID'])['acc_id1_nunique'].agg({'max_acc_id1_nunique_num_2': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left") 
    
    tmp = df.groupby(["device1"])["UID"].nunique().reset_index()
    tmp.columns =["device1","device1_nunique"]
    tmp.sort_values(by="device1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device1"]].drop_duplicates(),tmp,on="device1",how="left")[["UID","device1_nunique"]]
    tmp = tmp.groupby(['UID'])['device1_nunique'].agg({'max_device1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["mac1"])["UID"].nunique().reset_index()
    tmp.columns =["mac1","mac1_nunique"]
    tmp.sort_values(by="mac1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","mac1"]].drop_duplicates(),tmp,on="mac1",how="left")[["UID","mac1_nunique"]]
    tmp = tmp.groupby(['UID'])['mac1_nunique'].agg({'max_mac1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
     
    return res
    
def gen_leak_1(df):
    res = df[["UID"]].drop_duplicates()
    
    tmp = df.groupby(["ip1"])["UID"].nunique().reset_index()
    tmp.columns =["ip1","ip1_nunique"]
    tmp.sort_values(by="ip1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","ip1"]].drop_duplicates(),tmp,on="ip1",how="left")[["UID","ip1_nunique"]]
    tmp = tmp.groupby(['UID'])['ip1_nunique'].agg({'max_ip1_nunique_num': 'max','std_ip1_nunique_num': 'std'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left") 
    
    tmp = df.groupby(["ip2"])["UID"].nunique().reset_index()
    tmp.columns =["ip2","ip2_nunique"]
    tmp.sort_values(by="ip2_nunique",ascending=False)
    tmp = pd.merge(df[["UID","ip2"]].drop_duplicates(),tmp,on="ip2",how="left")[["UID","ip2_nunique"]]
    tmp = tmp.groupby(['UID'])['ip2_nunique'].agg({'max_ip2_nunique_num': 'max','std_ip2_nunique_num': 'std'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["ip1","ip2"])["UID"].nunique().reset_index()
    tmp.columns =["ip1","ip2","ip1_ip2_nunique"]
    tmp.sort_values(by="ip1_ip2_nunique",ascending=False)
    tmp = pd.merge(df[["UID","ip1","ip2"]].drop_duplicates(),tmp,on="ip2",how="left")[["UID","ip1_ip2_nunique"]]
    tmp = tmp.groupby(['UID'])['ip1_ip2_nunique'].agg({'max_ip1_ip2_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    
    
    tmp = df.groupby(["ip1_sub"])["UID"].nunique().reset_index()
    tmp.columns =["ip1_sub","ip1_sub_nunique"]
    tmp.sort_values(by="ip1_sub_nunique",ascending=False)
    tmp = pd.merge(df[["UID","ip1_sub"]].drop_duplicates(),tmp,on="ip1_sub",how="left")[["UID","ip1_sub_nunique"]]
    tmp = tmp.groupby(['UID'])['ip1_sub_nunique'].agg({'max_ip1_sub_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["ip2_sub"])["UID"].nunique().reset_index()
    tmp.columns =["ip2_sub","ip2_sub_nunique"]
    tmp.sort_values(by="ip2_sub_nunique",ascending=False)
    tmp = pd.merge(df[["UID","ip2_sub"]].drop_duplicates(),tmp,on="ip2_sub",how="left")[["UID","ip2_sub_nunique"]]
    tmp = tmp.groupby(['UID'])['ip2_sub_nunique'].agg({'max_ip2_sub_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device_code1"])["geo_code"].nunique().reset_index()
    tmp.columns =["device_code1","device_code1_nunique"]
    tmp.sort_values(by="device_code1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code1"]].drop_duplicates(),tmp,on="device_code1",how="left")[["UID","device_code1_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code1_nunique'].agg({'max_device_code1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device_code1"])["ip1"].nunique().reset_index()
    tmp.columns =["device_code1","device_code1_nunique"]
    tmp.sort_values(by="device_code1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code1"]].drop_duplicates(),tmp,on="device_code1",how="left")[["UID","device_code1_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code1_nunique'].agg({'max_device_code1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    

    tmp = df.groupby(["device_code1"])["UID"].nunique().reset_index()
    tmp.columns =["device_code1","device_code1_nunique"]
    tmp.sort_values(by="device_code1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code1"]].drop_duplicates(),tmp,on="device_code1",how="left")[["UID","device_code1_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code1_nunique'].agg({'max_device_code1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device_code2"])["UID"].nunique().reset_index()
    tmp.columns =["device_code2","device_code2_nunique"]
    tmp.sort_values(by="device_code2_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code2"]].drop_duplicates(),tmp,on="device_code2",how="left")[["UID","device_code2_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code2_nunique'].agg({'max_device_code2_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device_code3"])["UID"].nunique().reset_index()
    tmp.columns =["device_code3","device_code3_nunique"]
    tmp.sort_values(by="device_code3_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device_code3"]].drop_duplicates(),tmp,on="device_code3",how="left")[["UID","device_code3_nunique"]]
    tmp = tmp.groupby(['UID'])['device_code3_nunique'].agg({'max_device_code3_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    tmp = df.groupby(["device1"])["UID"].nunique().reset_index()
    tmp.columns =["device1","device1_nunique"]
    tmp.sort_values(by="device1_nunique",ascending=False)
    tmp = pd.merge(df[["UID","device1"]].drop_duplicates(),tmp,on="device1",how="left")[["UID","device1_nunique"]]
    tmp = tmp.groupby(['UID'])['device1_nunique'].agg({'max_device1_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    
    tmp = df.groupby(["mac2"])["UID"].nunique().reset_index()
    tmp.columns =["mac2","mac2_nunique"]
    tmp.sort_values(by="mac2_nunique",ascending=False)
    tmp = pd.merge(df[["UID","mac2"]].drop_duplicates(),tmp,on="mac2",how="left")[["UID","mac2_nunique"]]
    tmp = tmp.groupby(['UID'])['mac2_nunique'].agg({'max_mac2_nunique_num': 'max'}).reset_index().fillna(0)
    res = pd.merge(res,tmp,on="UID",how="left")
    
    print(res.head(5))
    
    return res

def feature_encoding(df_label,op_train,op_test,train,test,col):
    op_train = pd.merge(op_train,df_label,on="UID",how="left")
    for feature in col:
        print(op_train.columns)
        tmp = analyse(op_train,[feature],label='Tag')
        tmp = tmp.sort_values(by=feature,ascending=False)
        print(tmp.head(10))
        xx_train = pd.merge(op_train,tmp,on=feature,how="left")
        xx_test = pd.merge(op_test,tmp,on=feature,how="left")
        print(xx_train.columns)
        tmp = xx_train.groupby(['UID'])['rate'].agg({
                                                                 'median_rate': 'median',
                                                                 'max_rate': 'max',
                                                                }).reset_index().fillna(0)
        train = pd.merge(train,tmp,on="UID",how="left")
        
        tmp = xx_test.groupby(['UID'])['rate'].agg({
                                                                 'median_rate': 'median',
                                                                 'max_rate': 'max',
                                                                }).reset_index().fillna(0)
        test = pd.merge(test,tmp,on="UID",how="left")
    del op_train["Tag"]
    
    
        

    return train,test



def get_w2v_fea(op_train, op_test, train, test,i):
    df = pd.read_csv("../data/new/w2v/merchantnew.csv")
    df = df.drop_duplicates([i])
    op_train = op_train[["UID", i]]
    op_test = op_test[["UID", i]]
    op_train = pd.merge(op_train, df, on=i, how='left')
    op_test = pd.merge(op_test, df, on=i, how='left')
    col = [x for x in op_train.columns if "%sW"%i in x]
    for feature in col:
        tmp = op_train.groupby(['UID'])[feature].agg({
            'median_rate': 'mean',
        }).reset_index().fillna(0)
        train = pd.merge(train, tmp, on="UID", how="left")

        tmp = op_test.groupby(['UID'])[feature].agg({
            'median_rate': 'mean',
        }).reset_index().fillna(0)
        test = pd.merge(test, tmp, on="UID", how="left")

    return train, test


def get_w2v_fea3(op_train, op_test, train, test,i):
    df = pd.read_csv("../data/new/w2v/xxnew_.csv")
    df = df.drop_duplicates([i])
    op_train["xx"] = op_train["day"].astype(str) + op_train["time"].apply(lambda x:"".join(x.split(":")[:2]))
    op_test["xx"] = op_test["day"].astype(str) + op_test["time"].apply(lambda x:"".join(x.split(":")[:2]))
    op_train["xx"] = op_train["xx"].astype(int)
    op_test["xx"] = op_test["xx"].astype(int)
    op_train = op_train[["UID", i]]
    op_test = op_test[["UID", i]]
    
    print(op_train.dtypes)
    print(df.dtypes)
    op_train = pd.merge(op_train, df, on=i, how='left')
    op_test = pd.merge(op_test, df, on=i, how='left')
    col = [x for x in op_train.columns if "%sW"%i in x]
    for feature in col:
        tmp = op_train.groupby(['UID'])[feature].agg({
            'median_rate': 'mean',
        }).reset_index().fillna(0)
        train = pd.merge(train, tmp, on="UID", how="left")

        tmp = op_test.groupby(['UID'])[feature].agg({
            'median_rate': 'mean',
        }).reset_index().fillna(0)
        test = pd.merge(test, tmp, on="UID", how="left")

    return train, test

  

In [4]:
from sklearn.model_selection import StratifiedKFold
op_train = df_train_operation
op_test =df_test_operation
trans_train = df_train_transaction
trans_test = df_test_transaction
y = df_label


train = get_feature(op_train,trans_train,y).fillna(-1)
test = get_feature(op_test,trans_test,sub).fillna(-1)

df_train_leak_1 = gen_leak_1(df_train_operation)
df_train_leak_2 = gen_leak_2(df_train_transaction)
train = pd.merge(train,df_train_leak_1,on="UID",how="left")
train = pd.merge(train,df_train_leak_2,on="UID",how="left")

df_test_leak_1 = gen_leak_1(df_test_operation)
df_test_leak_2 = gen_leak_2(df_test_transaction)
test = pd.merge(test,df_test_leak_1,on="UID",how="left")
test = pd.merge(test,df_test_leak_2,on="UID",how="left")


print(train.shape)
print(test.shape)
train = train.drop(['Tag'],axis = 1).fillna(-1)
label = y['Tag']

test_id = test['UID']


day
trans_amt
bal
trans_type2
market_type


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:128: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:135: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


df shape (31179, 141)
day
trans_amt
bal
trans_type2
market_type
df shape (31588, 138)


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:318: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:326: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:333: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:342: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:349: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:364: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future 

     UID  max_ip1_nunique_num  std_ip1_nunique_num  max_ip2_nunique_num  \
0  10035                 32.0            13.954471                  0.0   
1  16264                  1.0             0.000000                  1.0   
2  13162                  2.0             0.447214                  0.0   
3  21392                  2.0             0.333333                  0.0   
4  18599                220.0            27.270864                  0.0   

   std_ip2_nunique_num  max_ip1_ip2_nunique_num  max_ip1_sub_nunique_num  \
0                  0.0                      0.0                    134.0   
1                  0.0                      0.0                     16.0   
2                  0.0                      0.0                     47.0   
3                  0.0                      0.0                     32.0   
4                  0.0                      0.0                    220.0   

   max_ip2_sub_nunique_num  max_device_code1_nunique_num_x  \
0                      0.0    

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:218: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:227: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:234: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:241: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:248: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:256: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future 

      UID  max_ip1_nunique_num  std_ip1_nunique_num  max_ip2_nunique_num  \
0  119982                  2.0             0.500000                  0.0   
1  129994                  1.0             0.000000                  0.0   
2  112135                 94.0            44.477522                  0.0   
3  121943                  1.0             0.000000                  0.0   
4  114176                  1.0             0.000000                  0.0   

   std_ip2_nunique_num  max_ip1_ip2_nunique_num  max_ip1_sub_nunique_num  \
0                  0.0                      0.0                      9.0   
1                  0.0                      0.0                     14.0   
2                  0.0                      0.0                    136.0   
3                  0.0                      0.0                      7.0   
4                  0.0                      0.0                     24.0   

   max_ip2_sub_nunique_num  max_device_code1_nunique_num_x  \
0                      0

In [ ]:
print(train.shape,test.shape)
trans_train["merchant_2"] = trans_train["merchant"].fillna("missing").apply(lambda x:x[:2])
trans_test["merchant_2"] = trans_test["merchant"].fillna("missing").apply(lambda x:x[:2])

op_train["geo_2"] = op_train["geo_code"].fillna("missing").apply(lambda x:x[:2])
op_test["geo_2"] = op_test["geo_code"].fillna("missing").apply(lambda x:x[:2])
trans_train["geo_2"] = trans_train["geo_code"].fillna("missing").apply(lambda x:x[:2])
trans_test["geo_2"] = trans_test["geo_code"].fillna("missing").apply(lambda x:x[:2])

train,test = feature_encoding(df_label,op_train,op_test,train,test,["os","version","geo_2"])
train,test = feature_encoding(df_label,trans_train,trans_test,train,test,["channel","merchant_2","geo_2"])
print(train.shape,test.shape)
# (31179, 178) (31198, 179)

(31179, 167) (31588, 165)
Index(['UID', 'day', 'mode', 'success', 'time', 'os', 'version', 'device1',
       'device2', 'device_code1', 'device_code2', 'device_code3', 'mac1',
       'mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub',
       'ip1_sup_add_ip2_sub', 'os_version', 'null_sum', 'geo_2', 'device2_2',
       'Tag'],
      dtype='object')


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if sys.path[0] == '':


    os  count   sum      rate
6  200  16700  2857  0.170873
5  107     83     2  0.019417
4  105      4     4  0.166667
3  104    179    25  0.125628
2  103   6297  2188  0.346367
1  102  24054  2769  0.115020
0  101   4211  1077  0.254550
Index(['UID', 'day', 'mode', 'success', 'time', 'os', 'version', 'device1',
       'device2', 'device_code1', 'device_code2', 'device_code3', 'mac1',
       'mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub',
       'ip1_sup_add_ip2_sub', 'os_version', 'null_sum', 'geo_2', 'device2_2',
       'Tag', 'count', 'sum', 'rate'],
      dtype='object')


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:439: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:445: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


Index(['UID', 'day', 'mode', 'success', 'time', 'os', 'version', 'device1',
       'device2', 'device_code1', 'device_code2', 'device_code3', 'mac1',
       'mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub',
       'ip1_sup_add_ip2_sub', 'os_version', 'null_sum', 'geo_2', 'device2_2',
       'Tag'],
      dtype='object')
   version  count   sum      rate
37   7.1.0      3     0  0.000000
36   7.0.9  17432  1637  0.093800
35   7.0.8      1     0  0.000000
34   7.0.7   4004   927  0.230368
33   7.0.6      2     0  0.000000
32   7.0.5  16753  2030  0.121028
31   7.0.2   3463   357  0.102498
30   7.0.1     62     1  0.012195
29   7.0.0   1020   712  0.684615
28   6.6.3    731   513  0.683089


In [ ]:
print(train.shape)
print(test.shape)
train,test = get_w2v_fea(trans_train,trans_test,train,test,"merchant")
print(train.shape)
print(test.shape)

In [ ]:
print([i for i in test.columns if i  not in train.columns])
print([i for i in train.columns if i  not in test.columns])

train = train.drop(['UID','Tag',103.0, '3f469aa3836e71cb', '4adc3de71fe1a83c', '9d7dd7b80e806024', 'd9c417304a5ae70c'],axis = 1).fillna(-1)
test = test.drop(['UID','0c23319fa2efdd9f', '768160899ae359f6', 'd0c5bc6f28f5ad35'],axis = 1).fillna(-1)
print(train.shape)
print(test.shape)


In [ ]:
# lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=100, reg_alpha=3, reg_lambda=5, max_depth=6,
#     n_estimators=5000, objective="binary", subsample=0.9, colsample_bytree=0.77, subsample_freq=1, learning_rate=0.05,
#     random_state=1000, n_jobs=16, min_child_weight=4, min_child_samples=5, min_split_gain=0,class_weight={0:1,1:3})

lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=100, reg_alpha=3, reg_lambda=5, max_depth=-1,
    n_estimators=5000, objective="binary", subsample=0.9, colsample_bytree=0.77, subsample_freq=1, learning_rate=0.05,
    random_state=1000, n_jobs=16, min_child_weight=4, min_child_samples=5, min_split_gain=0)
skf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)

best_score = []

oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test_id.shape[0])

for index, (train_index, test_index) in enumerate(skf.split(train, label)):
    lgb_model.fit(train.iloc[train_index], label.iloc[train_index], verbose=50,
                  eval_set=[(train.iloc[train_index], label.iloc[train_index]),
                            (train.iloc[test_index], label.iloc[test_index])], early_stopping_rounds=30)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    oof_preds[test_index] = lgb_model.predict_proba(train.iloc[test_index], num_iteration=lgb_model.best_iteration_)[:,1]

    test_pred = lgb_model.predict_proba(test, num_iteration=lgb_model.best_iteration_)[:, 1]
    sub_preds += test_pred / 5
    #print('test mean:', test_pred.mean())
    #predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
    
m = tpr_weight_funtion(y_predict=oof_preds,y_true=label)
print(m[1])


In [13]:
m = tpr_weight_funtion(y_predict=oof_preds,y_true=label)
print(m[1])
sub['Tag'] = sub_preds
sub.to_csv('../submit/baseline_%s.csv'%str(m[1]),index=False)
# 0.8139556592765461

0.8417836498761354
